In [2]:
# IMPORTS
# data-related libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

# api-related libraries
import requests

In [3]:
# STEP 0 - INPUT CORRECT VARIABLES
home_team = "Nuggets"
away_team = "Pelicans"
current_date = "2023-01-31" # this is to get all games before this date



In [4]:
# STEP 1 - FETCH TEAMS
# keys: ['id', 'abbreviation', 'city', 'conference', 'division', 'full_name', 'name']
teams = (requests.get("https://www.balldontlie.io/api/v1/teams")).json()['data']

home_id = 0
away_id = 0

print('-------- ALL TEAMS --------')
for team in teams:
    print(team['name'])
    if (team['name'] == home_team): 
        home_id = team['id']
    if (team['name'] == away_team): 
        away_id = team['id']

print('-------- TEAM MATCHUP IDS --------')
print(home_team, home_id)
print(away_team, away_id)

-------- ALL TEAMS --------
Hawks
Celtics
Nets
Hornets
Bulls
Cavaliers
Mavericks
Nuggets
Pistons
Warriors
Rockets
Pacers
Clippers
Lakers
Grizzlies
Heat
Bucks
Timberwolves
Pelicans
Knicks
Thunder
Magic
76ers
Suns
Trail Blazers
Kings
Spurs
Raptors
Jazz
Wizards
-------- TEAM MATCHUP IDS --------
Nuggets 8
Pelicans 19


In [5]:
# STEP 2 - FIND ALL GAMES FOR THESE TEAMS
all_games_home_team = []
all_games_away_team = []


for page in range(0,10):
    games = (requests.get(f"https://www.balldontlie.io/api/v1/games?page={page+1}&seasons[]=2022&team_ids[]={home_id}&end_date={current_date}"))
    games.raise_for_status()
    all_games_home_team += games.json()['data']
    if (len(games.json()['data']) == 0): 
        break

    
for page in range(0,10):
    games = (requests.get(f"https://www.balldontlie.io/api/v1/games?page={page+1}&seasons[]=2022&team_ids[]={away_id}&end_date={current_date}"))
    games.raise_for_status()
    all_games_away_team += games.json()['data']
    if (len(games.json()['data']) == 0):
        break
    
print(f"-------- {home_team} All Games Before {current_date} --------")
print(len(all_games_home_team), 'games')
print(f"-------- {away_team} All Games Before {current_date} --------")
print(len(all_games_away_team), 'games')



-------- Nuggets All Games Before 2023-01-31 --------
51 games
-------- Pelicans All Games Before 2023-01-31 --------
52 games


In [6]:
# STEP 3a - GET STATS OF A GAME FOR EACH PLAYER ON HOME TEAM

home_team_stats_for_all_games = {}

for game in all_games_home_team:
    stats_for_game = (requests.get(f"https://www.balldontlie.io/api/v1/stats?game_ids[]={game['id']}", headers = {'User-agent': 'your bot 0.1'}))
    stats_for_game.raise_for_status()
    home_team_stats_for_all_games[game['id']] = stats_for_game.json()['data']

In [8]:
# STEP 3b - GET STATS OF A GAME FOR EACH PLAYER ON HOME TEAM

away_team_stats_for_all_games = {}

for game in all_games_away_team:
    stats_for_game = (requests.get(f"https://www.balldontlie.io/api/v1/stats?game_ids[]={game['id']}", headers = {'User-agent': 'your bot 0.1'}))
    stats_for_game.raise_for_status()
    away_team_stats_for_all_games[game['id']] = stats_for_game.json()['data']

In [9]:
# STEP 4a - FORMAT BASED ON PLAYERS IN HOME TEAM
home_team_player_stats = {}
for game_id in home_team_stats_for_all_games:
    for player_stat in home_team_stats_for_all_games[game_id]:
        if (player_stat['team']['name'] == home_team):
            first_name = player_stat['player']['first_name']
            last_name = player_stat['player']['last_name']
            full_name = first_name + " " + last_name
            if full_name in home_team_player_stats:
                home_team_player_stats[full_name] += [player_stat]
            else:
                home_team_player_stats[full_name] = [player_stat]


In [10]:
# STEP 4b - FORMAT BASED ON PLAYERS IN AWAY TEAM

away_team_player_stats = {}
for game_id in away_team_stats_for_all_games:
    for player_stat in away_team_stats_for_all_games[game_id]:
        if (player_stat['team']['name'] == away_team):
            first_name = player_stat['player']['first_name']
            last_name = player_stat['player']['last_name']
            full_name = first_name + " " + last_name
            if full_name in away_team_player_stats:
                away_team_player_stats[full_name] += [player_stat]
            else:
                away_team_player_stats[full_name] = [player_stat]


In [19]:
# STEP 5 - DECLARE THE PLAYER AND THE CONDITIONS YOU NEED FOR A SINGLE PLAYER
player_full_name = "Aaron Gordon"


visible_attributes = ["id", "date", "ast", "blk", "reb", "stl", "pts", "fg3a", "fg3m"]
df = None

if player_full_name in away_team_player_stats:
    df = pd.DataFrame(away_team_player_stats[player_full_name])
elif player_full_name in home_team_player_stats:
    df = pd.DataFrame(home_team_player_stats[player_full_name])

df['date'] = df['game'].apply(lambda x: x.get('date'))

df = df[visible_attributes]
sorted_df = df.sort_values(by="date", ascending=True)
filtered_df = df[(df['pts'] >= 10)]

print(f"{player_full_name} had {len(filtered_df)}/{len(df)} games with these parlays:")
print(sorted_df)

Aaron Gordon had 34/42 games with these parlays:
          id                      date  ast  blk  reb  stl  pts  fg3a  fg3m
18  12070543  2022-10-19T00:00:00.000Z    2    1   10    1   22     2     0
0   12069242  2022-10-21T00:00:00.000Z    3    0    3    0   10     6     0
1   12069293  2022-10-22T00:00:00.000Z    2    3   10    0   11     3     1
2   12069975  2022-10-24T00:00:00.000Z    1    0    6    0   26     2     0
3   12096541  2022-10-26T00:00:00.000Z    5    2    9    0    6     3     0
14  12160392  2022-10-28T00:00:00.000Z    0    2    1    0    3     3     1
7   12300133  2022-11-03T00:00:00.000Z    3    0    5    1   27     4     3
9   12309071  2022-11-05T00:00:00.000Z    2    0    6    0    8     1     0
10  12317286  2022-11-07T00:00:00.000Z    5    1    5    4    9     1     0
16  12323319  2022-11-09T00:00:00.000Z    6    1   16    1   18     3     1
11  12341060  2022-11-13T00:00:00.000Z    2    0    5    0   13     1     1
4   12353812  2022-11-16T00:00:00.000Z 